In [255]:
import os
import pandas as pd
import numpy as np
import math
from datetime import datetime
import datetime as dt
import tensorflow as tf
from tensorflow import keras
from keras.layers import GRU, Dropout, SimpleRNN, LSTM, Dense, SimpleRNN, GRU
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

import plotly as py
import plotly.io as pio

# Mute sklearn warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

from plotly.offline import init_notebook_mode

# Show charts when running kernel
init_notebook_mode(connected=True)

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import seaborn as sns
import matplotlib.pyplot as plt
from colorama import Fore

In [256]:
# Path to your CSV file
csv_file_path = '/Users/rayhanzirvi/Desktop/acs_5yr_est_selected_economic_characteristics_2010-2022.csv'
# csv_file_path = 'math-ysics/SpringData2024/data/consume-produce_data/annual_generation_state.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# for col in df.columns:
#     print(col)

# Display the first few rows of the DataFrame
print(df.head())

# df_good = df[['State', '2021']]

# print(df_2021)



               Label (Grouping)           Category    State   Estimate  \
0             EMPLOYMENT STATUS             Header  Alabama        NaN   
1  Population 16 years and over  EMPLOYMENT STATUS  Alabama  3,714,504   
2                In labor force  EMPLOYMENT STATUS  Alabama  2,246,848   
3          Civilian labor force  EMPLOYMENT STATUS  Alabama  2,230,879   
4                      Employed  EMPLOYMENT STATUS  Alabama  2,036,867   

  Margin of Error    Percent Percent Margin of Error  Year  
0             NaN        NaN                     NaN  2010  
1          ±1,588  3,714,504                     (X)  2010  
2          ±5,576      60.5%                    ±0.1  2010  
3          ±5,531      60.1%                    ±0.1  2010  
4          ±5,904      54.8%                    ±0.2  2010  


In [257]:
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.lstrip()

In [258]:
employed = df[(df['Year'] == 2021) & (df['Label (Grouping)'] == 'Employed') & (df['Category'] == 'EMPLOYMENT STATUS')]
# Now, drop duplicates based on 'State', keeping only the first occurrence
employed = employed.drop_duplicates(subset='State', keep='first')

employed = employed[['State', 'Estimate', 'Margin of Error', 'Percent', 'Year']]

employed.head()

# percent is out of TOTAL POPULATION

,State,Estimate,Margin of Error,Percent,Year
90484,Alabama,"2,173,597","±8,587",54.3%,2021
90629,Alaska,"338,664","±2,500",59.2%,2021
90774,Arizona,"3,210,791","±7,197",56.8%,2021
90919,Arkansas,"1,310,863","±6,350",55.0%,2021
91064,California,"18,676,721","±21,726",59.3%,2021


In [259]:
unemployed = df[(df['Year'] == 2021) & (df['Label (Grouping)'] == 'Unemployed') & (df['Category'] == 'EMPLOYMENT STATUS')]
# Now, drop duplicates based on 'State', keeping only the first occurrence
# unemployed_update = unemployed.drop_duplicates(subset='State', keep='first')
unemployed = unemployed[['State', 'Estimate', 'Margin of Error', 'Percent', 'Year']]

unemployed.head()

# percent is out of TOTAL POPULATION

,State,Estimate,Margin of Error,Percent,Year
90485,Alabama,"124,416","±3,443",3.1%,2021
90630,Alaska,"25,054","±1,280",4.4%,2021
90775,Arizona,"191,115","±3,619",3.4%,2021
90920,Arkansas,"73,733","±2,564",3.1%,2021
91065,California,"1,303,741","±10,259",4.1%,2021


In [260]:
unemployed['Percent'] = unemployed['Percent'].str.rstrip('%').astype('float')

unemployed.head()

,State,Estimate,Margin of Error,Percent,Year
90485,Alabama,"124,416","±3,443",3.1,2021
90630,Alaska,"25,054","±1,280",4.4,2021
90775,Arizona,"191,115","±3,619",3.4,2021
90920,Arkansas,"73,733","±2,564",3.1,2021
91065,California,"1,303,741","±10,259",4.1,2021


In [261]:
employed['Percent'] = employed['Percent'].str.rstrip('%').astype('float')

employed.head()

,State,Estimate,Margin of Error,Percent,Year
90484,Alabama,"2,173,597","±8,587",54.3,2021
90629,Alaska,"338,664","±2,500",59.2,2021
90774,Arizona,"3,210,791","±7,197",56.8,2021
90919,Arkansas,"1,310,863","±6,350",55.0,2021
91064,California,"18,676,721","±21,726",59.3,2021


In [262]:
employed.set_index('State', inplace=True)
unemployed.set_index('State', inplace=True)

In [263]:
percent_unemploy = unemployed['Percent'] / (unemployed['Percent'] + employed['Percent']) * 100

percent_unemploy.head()

State
Alabama       5.400697
Alaska        6.918239
Arizona       5.647841
Arkansas      5.335628
California    6.466877
Name: Percent, dtype: float64

In [264]:
mean_income = df[(df['Year'] == 2021) & (df['Label (Grouping)'] == 'Mean household income (dollars)')]
# Now, drop duplicates based on 'State', keeping only the first occurrence
mean_income = mean_income.drop_duplicates(subset='State', keep='first')

mean_income = mean_income[['State', 'Estimate', 'Margin of Error', 'Percent', 'Year']]

mean_income.head()

,State,Estimate,Margin of Error,Percent,Year
90548,Alabama,"76,150",±467,(X),2021
90693,Alaska,"103,258","±1,045",(X),2021
90838,Arizona,"89,693",±503,(X),2021
90983,Arkansas,"73,346",±765,(X),2021
91128,California,"119,149",±348,(X),2021


In [265]:
median_income = df[(df['Year'] == 2021) & (df['Label (Grouping)'] == 'Median household income (dollars)')]
# Now, drop duplicates based on 'State', keeping only the first occurrence
median_income = median_income.drop_duplicates(subset='State', keep='first')

median_income = median_income[['State', 'Estimate', 'Margin of Error', 'Percent', 'Year']]

median_income.head()

,State,Estimate,Margin of Error,Percent,Year
90547,Alabama,"54,943",±377,(X),2021
90692,Alaska,"80,287","±1,113",(X),2021
90837,Arizona,"65,913",±387,(X),2021
90982,Arkansas,"52,123",±458,(X),2021
91127,California,"84,097",±236,(X),2021


In [266]:
per_capita_income = df[(df['Year'] == 2021) & (df['Label (Grouping)'] == 'Per capita income (dollars)')]
# Now, drop duplicates based on 'State', keeping only the first occurrence
per_capita_income = per_capita_income.drop_duplicates(subset='State', keep='first')

per_capita_income = per_capita_income[['State', 'Estimate', 'Year']]

per_capita_income.head()

,State,Estimate,Year
90573,Alabama,"30,458",2021
90718,Alaska,"39,236",2021
90863,Arizona,"34,644",2021
91008,Arkansas,"29,210",2021
91153,California,"41,276",2021


In [267]:
carpooling = df[(df['Year'] == 2021) & (df['Label (Grouping)'] == 'Car, truck, or van -- carpooled')]
# Now, drop duplicates based on 'State', keeping only the first occurrence
# carpooling = carpooling.drop_duplicates(subset='State', keep='first')

carpooling = carpooling[['State', 'Estimate', 'Year']]

carpooling.head()

,State,Estimate,Year
90501,Alabama,"177,167",2021
90646,Alaska,"41,473",2021
90791,Arizona,"325,297",2021
90936,Arkansas,"129,053",2021
91081,California,"1,756,016",2021


In [268]:
work = df[(df['Year'] == 2021) & (df['Label (Grouping)'] == 'Mean travel time to work (minutes)')]
# Now, drop duplicates based on 'State', keeping only the first occurrence
# work = work.drop_duplicates(subset='State', keep='first')

work = work[['State', 'Estimate', 'Year']]

work.head()

,State,Estimate,Year
90506,Alabama,25.2,2021
90651,Alaska,19.5,2021
90796,Arizona,25.7,2021
90941,Arkansas,22.0,2021
91086,California,29.5,2021


In [269]:
poverty_percent = df[(df['Year'] == 2021) & (df['Label (Grouping)'] == 'All people')]
# Now, drop duplicates based on 'State', keeping only the first occurrence
# poverty_percent = poverty_percent.drop_duplicates(subset='State', keep='first')

poverty_percent = poverty_percent[['State', 'Percent', 'Year']]

poverty_percent.head()

,State,Percent,Year
90615,Alabama,15.8%,2021
90760,Alaska,10.4%,2021
90905,Arizona,13.5%,2021
91050,Arkansas,16.0%,2021
91195,California,12.3%,2021


In [270]:
# Convert 'Percent' in df_poverty to numeric format, removing '%' and converting to float
poverty_percent['Percent'] = poverty_percent['Percent'].str.rstrip('%').astype(float)

# Merge the dataframes on 'State' and 'Year'
df_combined = pd.merge(per_capita_income, poverty_percent, on=['State', 'Year'], suffixes=('_income', '_poverty'))

# Rename columns
df_combined.rename(columns={'Estimate': 'Per_Capita_Income', 'Percent': 'Poverty_Percentage'}, inplace=True)

df_combined.head()

,State,Per_Capita_Income,Year,Poverty_Percentage
0,Alabama,"30,458",2021,15.8
1,Alaska,"39,236",2021,10.4
2,Arizona,"34,644",2021,13.5
3,Arkansas,"29,210",2021,16.0
4,California,"41,276",2021,12.3


In [271]:
# Merge the dataframes on 'State' and 'Year'
df_combined2 = pd.merge(df_combined, percent_unemploy, on=['State'], suffixes=('_income', '_poverty'))

# Rename columns
df_combined2.rename(columns={'Percent': 'Unemployment_Rate'}, inplace=True)

df_combined2.head()

,State,Per_Capita_Income,Year,Poverty_Percentage,Unemployment_Rate
0,Alabama,"30,458",2021,15.8,5.400697
1,Alaska,"39,236",2021,10.4,6.918239
2,Arizona,"34,644",2021,13.5,5.647841
3,Arkansas,"29,210",2021,16.0,5.335628
4,California,"41,276",2021,12.3,6.466877


In [272]:
# Merge the dataframes on 'State' and 'Year'
df_combined2 = pd.merge(df_combined2, carpooling, on=['State'], suffixes=('_', '_carpool'))

# Rename columns
df_combined2.rename(columns={'Estimate': 'Carpool_Estimate'}, inplace=True)

df_combined2.head()

,State,Per_Capita_Income,Year_,Poverty_Percentage,Unemployment_Rate,Carpool_Estimate,Year_carpool
0,Alabama,"30,458",2021,15.8,5.400697,"177,167",2021
1,Alaska,"39,236",2021,10.4,6.918239,"41,473",2021
2,Arizona,"34,644",2021,13.5,5.647841,"325,297",2021
3,Arkansas,"29,210",2021,16.0,5.335628,"129,053",2021
4,California,"41,276",2021,12.3,6.466877,"1,756,016",2021


In [273]:
df_combined2 = df_combined2.drop('Year_', axis=1)

df_combined2 = df_combined2.drop('Year_carpool', axis=1)

df_combined2.head()

,State,Per_Capita_Income,Poverty_Percentage,Unemployment_Rate,Carpool_Estimate
0,Alabama,"30,458",15.8,5.400697,"177,167"
1,Alaska,"39,236",10.4,6.918239,"41,473"
2,Arizona,"34,644",13.5,5.647841,"325,297"
3,Arkansas,"29,210",16.0,5.335628,"129,053"
4,California,"41,276",12.3,6.466877,"1,756,016"


In [274]:
# Merge the dataframes on 'State' and 'Year'
df_combined2 = pd.merge(df_combined2, work, on=['State'], suffixes=('_', '_work'))

# Rename columns
df_combined2.rename(columns={'Estimate': 'Work_Estimate'}, inplace=True)

df_combined2.head()

,State,Per_Capita_Income,Poverty_Percentage,Unemployment_Rate,Carpool_Estimate,Work_Estimate,Year
0,Alabama,"30,458",15.8,5.400697,"177,167",25.2,2021
1,Alaska,"39,236",10.4,6.918239,"41,473",19.5,2021
2,Arizona,"34,644",13.5,5.647841,"325,297",25.7,2021
3,Arkansas,"29,210",16.0,5.335628,"129,053",22.0,2021
4,California,"41,276",12.3,6.466877,"1,756,016",29.5,2021


In [275]:
df_combined2 = df_combined2.drop('Year', axis=1)

df_combined2.head()

,State,Per_Capita_Income,Poverty_Percentage,Unemployment_Rate,Carpool_Estimate,Work_Estimate
0,Alabama,"30,458",15.8,5.400697,"177,167",25.2
1,Alaska,"39,236",10.4,6.918239,"41,473",19.5
2,Arizona,"34,644",13.5,5.647841,"325,297",25.7
3,Arkansas,"29,210",16.0,5.335628,"129,053",22.0
4,California,"41,276",12.3,6.466877,"1,756,016",29.5


In [276]:
# Path to your CSV file
csv_file_path = '/Users/rayhanzirvi/Desktop/ACSST1Y2021.S1501-Data.csv'
# csv_file_path = 'math-ysics/SpringData2024/data/consume-produce_data/annual_generation_state.csv'

# Read the CSV file into a DataFrame
dp = pd.read_csv(csv_file_path)

# for col in df.columns:
#     print(col)

# Display the first few rows of the DataFrame
print(dp.head())

        GEO_ID                  NAME  \
0    Geography  Geographic Area Name   
1  0400000US01               Alabama   
2  0400000US02                Alaska   
3  0400000US04               Arizona   
4  0400000US05              Arkansas   

                                      S1501_C01_001E  \
0  Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT...   
1                                             468727   
2                                              67493   
3                                             681735   
4                                             284335   

                                      S1501_C01_001M  \
0  Margin of Error!!Total!!AGE BY EDUCATIONAL ATT...   
1                                               6283   
2                                               2489   
3                                               2668   
4                                               4778   

                                      S1501_C01_002E  \
0  Estimate!!Total!!AGE BY ED

In [277]:

# Now, drop duplicates based on 'State', keeping only the first occurrence
# edu = edu.drop_duplicates(subset='State', keep='first')

dp = dp[['NAME', 'S1501_C01_005E']]

dp = dp.iloc[1:]

dp.rename(columns={'NAME': 'State', 'S1501_C01_005E': 'Education_Estimate'}, inplace=True)

dp.head()

,State,Education_Estimate
1,Alabama,42643
2,Alaska,2377
3,Arizona,71571
4,Arkansas,27533
5,California,434774


In [278]:
# Merge the dataframes on 'State' and 'Year'
df_combined2 = pd.merge(df_combined2, dp, on=['State'], suffixes=('_', '_work'))

# Rename columns
# df_combined2.rename(columns={'Estimate': 'Work_Estimate'}, inplace=True)

df_combined2.head()

,State,Per_Capita_Income,Poverty_Percentage,Unemployment_Rate,Carpool_Estimate,Work_Estimate,Education_Estimate
0,Alabama,"30,458",15.8,5.400697,"177,167",25.2,42643
1,Alaska,"39,236",10.4,6.918239,"41,473",19.5,2377
2,Arizona,"34,644",13.5,5.647841,"325,297",25.7,71571
3,Arkansas,"29,210",16.0,5.335628,"129,053",22.0,27533
4,California,"41,276",12.3,6.466877,"1,756,016",29.5,434774


In [279]:
# State name to abbreviation mapping
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC',
}

# Use map() to change state names to abbreviations
df_combined2['State'] = df_combined2['State'].map(state_abbreviations)

df_combined2.head()

,State,Per_Capita_Income,Poverty_Percentage,Unemployment_Rate,Carpool_Estimate,Work_Estimate,Education_Estimate
0,AL,"30,458",15.8,5.400697,"177,167",25.2,42643
1,AK,"39,236",10.4,6.918239,"41,473",19.5,2377
2,AZ,"34,644",13.5,5.647841,"325,297",25.7,71571
3,AR,"29,210",16.0,5.335628,"129,053",22.0,27533
4,CA,"41,276",12.3,6.466877,"1,756,016",29.5,434774


In [280]:
# Export DataFrame to CSV
df_combined2.to_csv('/Users/rayhanzirvi/Desktop/socio_regress.csv', index=False)